In [4]:
import torch
def load_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    return checkpoint
    # print(checkpoint['optimizer_state_dict'])
    # print(checkpoint['model_state_dict'])

checkpoint_path = '/home/OHAPP/checkpoints/temp_checkpoint.pt'
checkpoint = load_checkpoint(checkpoint_path)

In [22]:
import torch
from peft import PeftModel, LoraConfig
from transformers import AutoModelForCausalLM

import os
os.environ["http_proxy"] = "http://127.0.0.1:15777"
os.environ["https_proxy"] = "http://127.0.0.1:15777"

def load_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    return checkpoint['model_state_dict']

def merge_lora_model(model_name, lora_checkpoint_path, output_path):
    # 加载基础模型
    base_model = AutoModelForCausalLM.from_pretrained(
        model_name, 
        trust_remote_code=True, 
        cache_dir="../model"
    )
    
    # 配置 LoRA
    peft_config = LoraConfig(
        task_type="CAUSAL_LM",
        r=8,
        lora_alpha=32,
        lora_dropout=0.1,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
    )
    
    # 创建 PeftModel
    peft_model = PeftModel(base_model, peft_config)
    
    # 加载 LoRA checkpoint
    lora_state_dict = load_checkpoint(lora_checkpoint_path)
    
    # 加载 LoRA 权重到 PeftModel
    peft_model.load_state_dict(lora_state_dict, strict=False)
    
    # 合并 LoRA 权重到基础模型
    merged_model = peft_model.merge_and_unload()
    
    # 保存合并后的模型
    merged_model.save_pretrained(output_path)
    
    print(f"合并后的模型已保存到: {output_path}")

# 使用示例
model_name = "deepseek-ai/DeepSeek-Coder-V2-Lite-Instruct"
lora_checkpoint_path = "checkpoints/temp_checkpoint.pt"
output_path = "merged/model"

merge_lora_model(model_name, lora_checkpoint_path, output_path)

KeyboardInterrupt: 

In [21]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="../model", trust_remote_code=True)

In [14]:
tokenizer = AutoTokenizer.from_pretrained("/home/CangjieLLM/model/models--deepseek-ai--DeepSeek-Coder-V2-Lite-Base/snapshots/ea9b066cee82f82906fdd58898cb3788b1c5d770", trust_remote_code=True)

In [16]:
tokenizer = AutoTokenizer.from_pretrained("merged/model", trust_remote_code=True)